In [ ]:
################## CPU  BASED ITERATION SEGMENTATION USING TRADITIONAL COMPUTER VISION ##############

In [ ]:
!pip install pillow
!pip install tabulate

In [ ]:
#UNISNSTALL EXISTING NUMPY MAHATOS AND RE-INSALL IT BEFORE RUNNING THIS ?make sure you have a numpy1.24 or above

In [ ]:
#!pip uninstall numpy mahotas -y     ## run this two times just to make sure

In [ ]:
#!pip uninstall numpy mahotas -y 

In [ ]:
#!pip install numpy mahotas


In [ ]:
# Import necessary libraries
import os
import cv2
import time
import shutil
import random
import inspect
#import imageio as im
import numpy as np
import mahotas as mh
from PIL import Image
from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#from skimage import measure, filters


In [ ]:
#!pip install --upgrade mahotas

In [ ]:
####################### EXCEPT FOR RAW DATA I ALWAYS USE FRESH DIRECTORIES #############################

In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/raw_files"
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"

In [ ]:
################### THE BELOW ARE TEST DIRECTORIES THEY CHANGE BASED ON TESTS  #########################

In [ ]:
experiment = "test3" # you can change this according to your needs. to access that data.
#current experiments are in test2

In [ ]:
real = f"{working_dir}/{experiment}/real_images"
#########################################################################################################

directories = {
    "normalized": f"{working_dir}/{experiment}/normalized_images",
    "cropped": f"{working_dir}/{experiment}/cropped_images",
    "npy": f"{working_dir}/{experiment}/pre_processing/npy",
    "originals": f"{working_dir}/{experiment}/pre_processing/originals",
    "masks": f"{working_dir}/{experiment}/pre_processing/masks",
    "test": f"{working_dir}/{experiment}/pre_processing/test",
    "s": f"{working_dir}/{experiment}/S",
    "crop_original": f"{working_dir}/{experiment}/pre_processing/crop_originals",
    "crop_masks": f"{working_dir}/{experiment}/pre_processing/crop_masks"
}

In [ ]:
###############################################functions
start_time = 0  # Define start_time in the global scope

def starttime():
    global start_time  # Use the global keyword to access the global start_time variable
    start_time = time.time()
    #hint: starttime() - To start timer.
    
def endtime():
    end_time = time.time()
    execution_time = end_time - start_time
    print(f"Execution time: {execution_time:.2f} seconds")
    #hint: endtime() - To end timer

def check(path):
    # Create the directory if it does not exist
    os.makedirs(path, exist_ok=True)

    # Remove the directory and all its contents
    shutil.rmtree(path)

    # Create a new empty directory
    os.mkdir(path)
    #hint: check(path) - To recreate a particular path
            
def heavycrop(test):
    starttime()
    for z in tqdm(sorted(os.listdir(test))):
        if z.endswith("tif"):
            # Read in the image
            img = mh.imread(os.path.join(test, z))

            # Calculate the number of crops in each dimension
            height, width = img.shape[:2]
            num_crops_y = height // 512
            num_crops_x = width // 512

            for i in range(num_crops_y):
                for j in range(num_crops_x):
                    # Crop the image
                    start_y = i * 512
                    start_x = j * 512
                    img_cropped = img[start_y:start_y+512, start_x:start_x+512]

                    # Create a new file name for the cropped image
                    file_name, file_ext = os.path.splitext(z)
                    new_file_name = f"{file_name}_{i}_{j}{file_ext}"

                    # Save only if cropped image has shape (512, 512)
                    if img_cropped.shape == (512, 512):
                        mh.imsave(os.path.join(test, new_file_name), img_cropped)
                    else:
                        print(f"Warning: Cropped image has unexpected shape {img_cropped.shape}")
                        
            endtime()
            # Remove original image file after cropping is done.
            os.remove(os.path.join(test,z))
            #hint: heavycrop(spath) - To heavy crop all images 512x512
            
def npyconversion(tif_dir, npy_path):
    tif_files = [f for f in os.listdir(tif_dir) if f.endswith('.tif')]
    tif_files.sort()
    data = []
    for tif_file in tqdm(tif_files):
        img = Image.open(os.path.join(tif_dir, tif_file))
        data.append(np.array(img))
    np.save(npy_path, data)
    #hint: npyconversion(path , npy + '/filename.npy' ) -To create NPY files
            
def a2bcopy(path1, path2):
    for z in tqdm(sorted(os.listdir(path1))):
        if z.endswith("tif"):
            shutil.copy(os.path.join(path1, z), os.path.join(path2, z))
            #hint: a2bcopy(sorce path, dest path) - To copy all images
            
def crop(test):
    for z in tqdm(sorted(os.listdir(test))):
        if (z.endswith("tif")): # checking the file ends with tif
            # Read in the image
            img = mh.imread(os.path.join(test, z))
            img_cropped = img[1000:2500, 2500:4500]
            mh.imsave(os.path.join(test, z), img_cropped)
            print(z)
            #hint: crop(spath) - To crop all images
    
def a2brandom(src_dir, dst_dir, number):
    # Get a list of all image files in the source directory
    image_files = [f for f in tqdm(os.listdir(src_dir)) if f.endswith('.tif')]
    # Randomly select 10 images from the list
    selected_images = random.sample(image_files, number)
    # Copy the selected images to the destination directory
    for image in selected_images:
        src_path = os.path.join(src_dir, image)
        dst_path = os.path.join(dst_dir, image)
        shutil.copy2(src_path, dst_path)
    # Print a message when done
    print('Copied 10 random images to', dst_dir)
    #hint: a2brandom(sorce path, dest path, random number) - To copy n random images
    
def count_files(dir_path):
    if os.path.isdir(dir_path):
        file_count = 0
        for file_name in os.listdir(dir_path):
            file_path = os.path.join(dir_path, file_name)
            file_size_bytes = os.path.getsize(file_path)
            file_size_mb = round(file_size_bytes / (1024 * 1024), 2)
            #print(f'{file_name} - Size: {file_size_mb} MB')
            file_count += 1
        return file_count
    else:
        print(f"{dir_path} is not a valid directory")
        return 0    
        #hint: count_files(path) - To count number of files in that path
    
def norm(path):
    for z in tqdm(sorted(os.listdir(path))):# added interactive progressbar to decrease the uncertanity and to increase curiosity :)
        if (z.endswith("tif")): # checking the file ends with tif 
            img = mh.imread(os.path.join(path, z))            
            # Normalize the image
            img = img.astype(np.float64)
            img /= img.max()
            img *= 255            
            # Save the processed image back to the temporary directory
            mh.imsave(os.path.join(path, z), img)
            #hint: norm(path) - To normalize all the images in the path
            
def shape(raw):
    for z in tqdm(sorted(os.listdir(raw))):
        if (z.endswith("tif")):
            img = mh.imread(os.path.join(raw, z))
            print (img.shape)
            #hint: shape(path) _ To print shape of all the images in the path
            
def refresh(experiment: str, directories: dict):
    for key in directories:
        if os.path.exists(directories[key]):
            shutil.rmtree(directories[key])
        os.makedirs(directories[key])
        #hint: refresh("experiment name", directories) - to recreate all directories in that dict
        
def paths(directories):
    for key, value in directories.items():
        globals()[key] = value
    return directories
    #hint: paths(directories) - To call the directories outside the dictionary
        
def help():
    functions = [value for key, value in globals().items() if inspect.isfunction(value)]
    headers = ["Function", "Hint", "Used for"]
    data = []
    for func in functions:
        source = inspect.getsource(func)
        lines = source.split("\n")
        hint_line = [line for line in lines if line.strip().startswith("#hint:")]
        if hint_line:
            hint_parts = hint_line[0].split("#hint:")[1].strip().split(" - ")
            hint_text = hint_parts[0]
            usage_text = hint_parts[1] if len(hint_parts) > 1 else ""
            data.append([f"{func.__name__}()", hint_text, usage_text])
    print(tabulate(data, headers=headers))
            
def check_all(directories):
    for key, path in directories.items():
        if os.path.exists(path):
            shutil.rmtree(path)
        os.makedirs(path)
        #hint: check_all(directories) - To create all directories fresh only use for new experiments.
        
def readpaths(directories):
    for key, path in directories.items():
        globals()[key] = path
        #hint: readpaths(directories) - To create all directories fresh only use for new experiments.
            
#def del(path):



In [ ]:
paths(directories)

In [ ]:
help()

In [ ]:
########## The below are manual functions to delete any particular directory and create it again. ##########

In [ ]:
#check_all(directories)

In [ ]:
# check(real)
# check(normalized)
# check(cropped)

# check(crop_originals)
# check(crop_masks)
# check(npy)
# check(originals)
# check(masks)
# check(test)

In [ ]:
#MANUALLY CHECK THAT ALL FOLDERS ARE AVAILABLE AND EMPTY

In [ ]:
####################      Getting some random data                ###############################
raw = '/raid/mpsych/RISTERLAB/VSOverviewTileSet/Acquired/'
raw1 = '/raid/mpsych/RISTERLAB/NINA_D1_MUTANTVSOverviewTileSet/Acquired'

In [ ]:
#a2brandom(raw,data,800)

In [ ]:
#a2brandom(raw1,data,2000)

In [ ]:
#count_files(data)

In [ ]:
#a2bcopy(data,real)

In [ ]:
#a2bcopy(data,normalized)

In [ ]:
#a2bcopy(data,test)

In [ ]:
#a2bcopy(data,cropped)

In [ ]:
#count_files(cropped)

In [ ]:
#############FINISHED COPYING FILES TO DIRECTORIES, NOW Normalization, cropping, segmentation ##############

In [ ]:
########## Normalization

In [ ]:
#norm(normalized)

In [ ]:
#norm(cropped)

In [ ]:
######## Cropping

In [ ]:
#crop(normalized)

In [ ]:
#crop(cropped)

In [ ]:
#crop(test)

In [ ]:
#heavycrop(test)

In [ ]:
#!lspci | grep -i nvidia
# TO check the graphic card availability

In [ ]:
#count_files(cropped)

In [ ]:
#################################    SEGMENTATION BEGIN    #########################################

In [ ]:
# success_count=0 
# for z in tqdm(sorted(os.listdir(cropped))):
#     if (z.endswith("tif")): # checking the file ends with tif 
#         img = im.imread(os.path.join(cropped, z))
#         print (z)
#         plt.imshow(img)
#         #plt.show()
    
#         # Apply a Gaussian filter to the image
#         c = img.copy()
#         #b = mh.gaussian_filter(b, sigma=3)

#         # Set values below 100 to 0
#         for a in range(150, 0, -1):
#             starttime()
#             b= img.copy()
#             b = mh.gaussian_filter(b, sigma=3)
#             b[b < a] = 0
#             #print (a)      
#             #b = exposure.equalize_hist(b)
#             # Label the regions in the filtered image
#             labeled, number = mh.label(b)


#             # filter based on labeled region size
#             sizes = mh.labeled.labeled_size(labeled)

#             # Remove the regions that are less than 1000
#             too_small = np.where(sizes < 1500)
#             labeled_only_big = mh.labeled.remove_regions(labeled, too_small)

#             #too_large = np.where(sizes > 20500)
#             #labeled_only_big = mh.labeled.remove_regions(labeled, too_large)
#             #for debug
#             #plt.imshow(labeled_only_big)
#             #plt.show()


#             # Create a binary mask from the filtered labeled regions
#             binary_mask = labeled_only_big.copy()
#             binary_mask[binary_mask > 0] = 1
#             labeled, number_1 = mh.label(binary_mask)


#              # Close the regions in the binary mask
#             binary_mask_closed = mh.morph.close(binary_mask)
            
            
#             plt.figure(figsize=(10,10))
#             #plt.imshow(binary_mask_closed)
#             #plt.show() 
            
#             # Set a threshold for the minimum region size           
#             min_region_size = 3000
            
#             # Initialize a variable to count the number of regions above the minimum size
#             large_regions = 0

#             # Get the sizes of the labeled regions
#             region_sizes = measure.regionprops(labeled, intensity_image=binary_mask_closed)

#             # Iterate over the region sizes and count the number of large regions
#             for region in region_sizes:
#                 if region.area > min_region_size:
#                      large_regions += 1


#             threshold = filters.threshold_otsu(binary_mask_closed) 
#             binary_image = binary_mask_closed > threshold
#             print('time taken for iteration',a,'image',z ,'is:')
#             endtime()
# #             if number_1>= 90:                
# #                 print (z)
# #                 plt.imshow(binary_image)
# #                 plt.show()
# #                 print (number_1)
# #                 print (threshold)
# #                 print(large_regions)
                  
#             if number_1 <=150 and number_1 >=100:
#                 if large_regions <=20:       # 20 is ideal value 
#                     print ("######################################################################")
#                     print(z)
#                     #plt.figure(figsize=(10,10))
#                     print("The image has clear segmentation.")
#                     #plt.imshow(binary_image)
#                     #plt.show()
#                     print (number_1)
#                     print (threshold)
#                     print(large_regions)
#                     shutil.move(os.path.join(normalized,z),os.path.join(originals,z))                 
#                     shutil.move(os.path.join(cropped,z),os.path.join(masks,z))                 
#                     mh.imsave(os.path.join(masks,z),binary_image)                 
#                     #print (sizes)
#                     print ("######################################################################")
#                     success_count+=1
#                     print (success_count)
#                     break
# print (success_count)
# print ('######################################### DONE        ############################################')

In [ ]:
# checking the files which got segemented well

In [ ]:
# origi = int(count_files(masks))
# croppi = int(count_files(real))
# seg_well =(((origi)/(croppi))*100) 
# print ("percentage = ",seg_well ,"%")

In [ ]:
count_files(masks)

In [ ]:
#print ("sucessfilly segemented", origi,"images")

In [ ]:
######################################   SEGMENTATION END   ################################################

In [ ]:
######copying data for heavycropping

In [ ]:
#a2bcopy(masks,crop_masks)

In [ ]:
#a2bcopy(originals,crop_originals)

In [ ]:
#heavycrop(crop_masks)

In [ ]:
#heavycrop(crop_originals)

In [ ]:
#count_files(crop_masks)

In [ ]:
#count_files(crop_originals)

In [ ]:
readpaths(directories)

In [ ]:
original = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test2/pre_processing/originals"
mask = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test2/pre_processing/masks"
npy1 = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test3/pre_processing/npy1"
pp="/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test3/post_processing/processed_prediction"

In [ ]:
ap="/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test3/post_processing/op"

In [ ]:
#count_files(pp)

In [ ]:
#count_files(crop_original)

In [ ]:
def a2bcopyforzinc(src,dst,c):
    for z in tqdm(sorted(os.listdir(c))):
        if (z.endswith("tif")):
            shutil.copy(os.path.join(src,z),os.path.join(dst,z))

In [ ]:
#a2bcopyforzinc(crop_original,ap,pp)

In [ ]:
#count_files(ap)

In [ ]:
#a2bcopy(original,crop_original)

In [ ]:
#a2bcopy(mask,crop_masks)

In [ ]:
def npycon(tif_dir, npy_path):
    tif_files = [f for f in os.listdir(tif_dir) if f.endswith('.tif')]
    data = []
    image_names = []
    for tif_file in tqdm(tif_files):
        img = Image.open(os.path.join(tif_dir, tif_file))
        data.append(np.array(img))
        image_names.append(tif_file)
    np.savez(npy_path, data=data, names=image_names)

In [ ]:
pom='/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test3/post_processing/data_population/old_mask'
poo ='/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test3/post_processing/data_population/old_original'
pno ='/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test3/post_processing/data_population/originals'
pnm ='/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/test3/post_processing/data_population/masks'

In [ ]:
#npycon(poo, npy1 + '/poo.npz')

In [ ]:
#npycon(pom, npy1 + '/pom.npz')

In [ ]:
npycon(pno, npy1 + '/pno.npz')

In [ ]:
npycon(pnm, npy1 + '/pnm.npz')

In [ ]:
#npycon(ap, npy1 + '/new_originals.npz' )

In [ ]:
#npycon(crop_original, npy1 + '/original.npz' )

In [ ]:
#npycon(crop_masks, npy1 + '/mask.npz')

In [ ]:
#npyconversion(crop_original, npy + '/original.npy' )

In [ ]:
#npyconversion(crop_masks, npy + '/mask.npy')

In [ ]:
#npyconversion(test, npy + '/test.npy' )

In [ ]:
#count_files(npy)

In [ ]:
##################### run this command


In [ ]:
#!pip install numpy==1.19.5

In [ ]:
########################################  KERAS  #########################################################

In [ ]:
#!pip install keras-unet

In [ ]:
import os
import cv2
import time
import shutil
import random
import inspect
#import imageio as im
import numpy as np
import mahotas as mh
from PIL import Image
from tabulate import tabulate
from tqdm.notebook import tqdm
import matplotlib.pyplot as plt
#from skimage import measure, filters

In [ ]:
# original directory path and working directory path for images
data = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir/raw_files"
working_dir = "/raid/mpsych/RISTERLAB/fly_segmentation_experiments/data_dir"
######################################################################################################
experiment = "test2" 
######################################################################################################
real = f"{working_dir}/{experiment}/real_images"
#########################################################################################################

directories = {
    "normalized": f"{working_dir}/{experiment}/normalized_images",
    "cropped": f"{working_dir}/{experiment}/cropped_images",
    "npy": f"{working_dir}/{experiment}/pre_processing/npy",
    "originals": f"{working_dir}/{experiment}/pre_processing/originals",
    "masks": f"{working_dir}/{experiment}/pre_processing/masks",
    "test": f"{working_dir}/{experiment}/pre_processing/test",
    "s": f"{working_dir}/{experiment}/S",
    "crop_original": f"{working_dir}/{experiment}/pre_processing/crop_originals",
    "crop_masks": f"{working_dir}/{experiment}/pre_processing/crop_masks"
}

In [ ]:
import keras
from keras_unet.models import custom_unet

In [ ]:
npy = f"{working_dir}/{experiment}/pre_processing/npy"

In [ ]:
images_file = os.path.join(npy, 'original.npy')
labels_file = os.path.join(npy, 'mask.npy')
#test_files = os.path.join(npy, 'test.npy')

In [ ]:
images = np.load(images_file)
labels = np.load(labels_file)
#test = np.load(test_file)

In [ ]:
images.shape

In [ ]:
 # full dataset does not have the last channel
images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)
#test = test.reshape(test.shape[0],test.shape[1],test.shape[2],1)

In [ ]:
images.shape

In [ ]:
p = np.random.permutation(len(images))

In [ ]:
images = images[p]
labels = labels[p]
#test = test[p]

In [ ]:
labels = labels.astype(np.float64)

In [ ]:
images = images.astype(np.float64)
for i in range(images.shape[0]):
    images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually


In [ ]:
images.shape

In [ ]:
X_train = images[0:1200]
y_train = labels[0:1200]
X_val = images[1201:1700]
y_val = labels[1201:1700]
X_test = images[1701:1812]
y_test = labels[1701:1812]

#X_test = test[80:]
#y_test = test[80:]

In [ ]:
model = custom_unet(
    input_shape=(512, 512, 1),
    use_batch_norm=False,
    num_classes=1,
    filters=32,
    dropout=0.5,
    output_activation='sigmoid')

In [ ]:
import keras.optimizers
from keras_unet.metrics import iou, iou_thresholded
from keras_unet.losses import jaccard_distance

In [ ]:
opt = keras.optimizer_v1.Adam(lr=0.01)

In [ ]:
model.compile(optimizer = 'Adam',    
              loss='binary_crossentropy', 
              metrics=[iou, iou_thresholded])

In [ ]:
##### batch size 50 , epochs =1000

In [ ]:
def training(batch,epochs):
    print ("training and prediction for","batch size:  ",batch, "epochs:  ", epochs)
    history = model.fit(X_train, 
                    y_train, 
                    batch, 
                    epochs=epochs,
                    validation_data=(X_val, y_val), 
                    verbose=1)
    
    from keras_unet.utils import plot_segm_history

    plot_segm_history(history)
    
    y_pred = model.predict(X_test)
    
    from keras_unet.utils import plot_imgs

    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=100)# number to plot also need to e defined in function
    

In [ ]:
# from keras import regularizers
# from keras.layers import Dense
# from keras.callbacks import EarlyStopping
# from keras.preprocessing.image import ImageDataGenerator
# from keras_unet.utils import plot_segm_history, plot_imgs

# def training(batch, epochs):
#     print("training and prediction for", "batch size: ", batch, "epochs: ", epochs)

# #     # Regularization
#     layer = Dense(units=64,
#                   kernel_regularizer=regularizers.l1_l2(l1=0.01, l2=0.01),
#                   bias_regularizer=regularizers.l2(0.01))

    # Early stopping
    #early_stopping = EarlyStopping(monitor='val_loss', patience=10)

#     # Data augmentation
#     datagen = ImageDataGenerator(rotation_range=20,
#                                  width_shift_range=0.2,
#                                  height_shift_range=0.2,
#                                  horizontal_flip=True)
#     datagen.fit(X_train)

#     # Training with data augmentation
#     history = model.fit(datagen.flow(X_train, y_train, batch_size=batch),
#                         steps_per_epoch=len(X_train) / batch,
#                         epochs=epochs,
#                         validation_data=(X_val, y_val),
#                         callbacks=[early_stopping])


########################################ONLY EARLY STOPPING###################################
#     # Early stopping
#     early_stopping = EarlyStopping(monitor='val_loss', patience=10)
#     # Training with early stopping
#     history = model.fit(X_train,
#                         y_train,
#                         batch_size=batch,
#                         epochs=epochs,
#                         validation_data=(X_val, y_val),
#                         callbacks=[early_stopping])

#     # Plotting results
#     plot_segm_history(history)
#     y_pred = model.predict(X_test)
#     plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=100)


In [ ]:
from keras import regularizers
from keras.layers import Dense
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator
from keras_unet.utils import plot_segm_history, plot_imgs

def trainingearlystopping(batch, epochs):
    print("training and prediction for", "batch size: ", batch, "epochs: ", epochs)
########################################ONLY EARLY STOPPING###################################
    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)
    # Training with early stopping
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Plotting results
    plot_segm_history(history)
    y_pred = model.predict(X_test)
    # Apply thresholding
    threshold = 0.2
    y_pred_binary = (y_pred > threshold).astype(np.uint8)
    plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=100)


In [ ]:
trainingearlystopping(60,100)

In [ ]:
trainingearlystopping(60,100)

In [ ]:
trainingearlystopping(60,100)

In [ ]:
############ TRYING THRESHOLD VALUES
import matplotlib.pyplot as plt

def trainingwiththres(batch, epochs, thresholds, n_images):
    print("training and prediction for", "batch size: ", batch, "epochs: ", epochs)

    # Early stopping
    early_stopping = EarlyStopping(monitor='val_loss', patience=10)

    # Training with early stopping
    history = model.fit(X_train,
                        y_train,
                        batch_size=batch,
                        epochs=epochs,
                        validation_data=(X_val, y_val),
                        callbacks=[early_stopping])

    # Plotting results
    plot_segm_history(history)
    y_pred = model.predict(X_test)

    # Apply multiple thresholds
    y_pred_binaries = []
    for threshold in thresholds:
        y_pred_binary = (y_pred > threshold).astype(np.uint8)
        y_pred_binaries.append(y_pred_binary)

    # Plot binary masks for multiple thresholds
    fig, axes = plt.subplots(n_images, len(thresholds), figsize=(15, 15))
    for i in range(n_images):
        for j in range(len(thresholds)):
            axes[i][j].imshow(y_pred_binaries[j][i])
            axes[i][j].set_title(f'Threshold {thresholds[j]}')
            axes[i][j].axis('off')
    plt.show()

In [ ]:
thresholds = [0.2, 0.4, 0.6, 0.8]
n_images = 10
history = trainingwiththres(60, 100, thresholds, n_images)

In [ ]:
#training(60,100)

In [ ]:

# a little overlifting has been noticed in the above graph so we are using new techiniques to decrease that

In [ ]:
training(60,100)

In [ ]:
history = model.fit(X_train, 
                    y_train, 
                    50, 
                    epochs=1000,
                    validation_data=(X_val, y_val), 
                    verbose=1)

In [ ]:
from keras_unet.utils import plot_segm_history

plot_segm_history(history)

In [ ]:
y_pred = model.predict(X_test)

In [ ]:
from keras_unet.utils import plot_imgs

plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)

In [ ]:
training(60,40)

In [ ]:
#help()

In [ ]:
training(65,65)

In [ ]:
overlay(directories['originals'],directories['masks'], 5)

In [ ]:
training(60,4000)

In [ ]:
training(64,20)

In [ ]:
import os
import random
from PIL import Image

def overlay(original_dir, mask_dir, n):
    original_images = os.listdir(original_dir)
    mask_images = os.listdir(mask_dir)

    for i in range(n):
        original_image = random.choice(original_images)
        mask_image = original_image.replace('.png', '_mask.png')
        if mask_image in mask_images:
            img = Image.open(os.path.join(original_dir, original_image))
            mask = Image.open(os.path.join(mask_dir, mask_image))
            img.paste(mask, (0, 0), mask)
            img.show()
            #hint: # overlay_masks('original', 'masks', 3) - To plot overlay wiht masks



In [ ]:
# !pip install keras-unet

# import keras
# from keras_unet.models import custom_unet

# images_file = os.path.join(npy_files, 'original.npy')
# labels_file = os.path.join(npy_files, 'mask.npy')

# images = np.load(images_file)
# labels = np.load(labels_file)

# images.shape

# # full dataset does not have the last channel
# images = images.reshape(images.shape[0],images.shape[1],images.shape[2],1)
# labels = labels.reshape(labels.shape[0],labels.shape[1],labels.shape[2],1)

# images.shape

# p = np.random.permutation(len(images))

# images = images[p]
# labels = labels[p]

# labels = labels.astype(np.float64)

# images = images.astype(np.float64)

# for i in range(images.shape[0]):
#     images[i] = (images[i] - images[i].min()) / (images[i].max() - images[i].min()) # normalize individually


# images.shape

# X_train = images[0:50]
# y_train = labels[0:50]
# X_val = images[10:20]
# y_val = labels[10:20]
# X_test = images[0:]
# y_test = labels[0:]

# model = custom_unet(
#     input_shape=(512, 512, 1),
#     use_batch_norm=False,
#     num_classes=1,
#     filters=32,
#     dropout=0.5,
#     output_activation='sigmoid')

# import keras.optimizers
# from keras_unet.metrics import iou, iou_thresholded
# from keras_unet.losses import jaccard_distance



# opt = keras.optimizer_v1.Adam(lr=0.01)

# model.compile(optimizer = 'Adam',    
#               loss='binary_crossentropy', 
#               metrics=[iou, iou_thresholded])

# history = model.fit(X_train, 
#                     y_train, 
#                     50, 
#                     epochs=500,
#                     validation_data=(X_val, y_val), 
#                     verbose=1)

# from keras_unet.utils import plot_segm_history

# plot_segm_history(history)

# y_pred = model.predict(X_test)

# from keras_unet.utils import plot_imgs

# plot_imgs(org_imgs=X_test, mask_imgs=y_test, pred_imgs=y_pred, nm_img_to_plot=10)

In [ ]:
# My next idea in the pipe line is to compare same images on different iterations and come to conclusion.
# To decide what batch size and epochs gives the best results.